In [1]:
import xarray as xr
import numpy as np
import netCDF4
import pandas as pd
from scipy import signal

In [2]:
var = "ahfs"
arr = []
for ens_index in range(1,17):
        filename = "/work/uo1075/u241321/data/u241321/data_cdo/ahfs_assi/"+var+"_Amon_MPI-ESM-LR_asSEIKERAf_r"+str(ens_index)+"i8p4_197001-201912_ym_g1d0.nc"
        arr.append(xr.load_dataset(filename,decode_times=False).assign_coords(ensemble=ens_index))

In [3]:
data = xr.concat(arr, dim="member")

In [4]:
annual_pre = data [var]

In [5]:
annual_prediction = annual_pre.transpose("time", "member", "lat", "lon") # 

In [19]:
annual_prediction

<xarray.DataArray 'ahfs' (time: 50, member: 16, lat: 180, lon: 360)>
array([[[[ 1.09617424e+01,  1.09619522e+01,  1.13102684e+01, ...,
           1.02593203e+01,  1.06139908e+01,  1.06141987e+01],
         [ 1.13434134e+01,  1.15446558e+01,  1.17259445e+01, ...,
           1.07973852e+01,  1.09709511e+01,  1.11521778e+01],
         [ 1.78647671e+01,  1.79905415e+01,  1.80816441e+01, ...,
           1.73977852e+01,  1.74917107e+01,  1.76984901e+01],
         ...,
         [-1.72125626e+00, -1.75214350e+00, -1.81290889e+00, ...,
          -1.61027241e+00, -1.66014397e+00, -1.69058979e+00],
         [-3.74166697e-01, -3.87063235e-01, -3.99661809e-01, ...,
          -2.99980015e-01, -3.18976223e-01, -3.51471037e-01],
         [-2.70299137e-01, -2.70317644e-01, -3.12162310e-01, ...,
          -1.94066405e-01, -2.42576510e-01, -2.42597237e-01]],

        [[ 1.10785847e+01,  1.10788012e+01,  1.14305868e+01, ...,
           1.03459463e+01,  1.07193623e+01,  1.07195807e+01],
         [ 1.14606581e+01,  1.16556005e+01,  1.18406334e+01, ...,
           1.08793869e+01,  1.10650206e+01,  1.12637987e+01],
         [ 1.77567825e+01,  1.79223785e+01,  1.80885487e+01, ...,
           1.74852028e+01,  1.75980473e+01,  1.76486721e+01],
...
          -4.15414572e+00, -4.16338491e+00, -4.14832163e+00],
         [-2.67452860e+00, -2.68894625e+00, -2.68934345e+00, ...,
          -2.59379864e+00, -2.62808347e+00, -2.65424085e+00],
         [-2.55855608e+00, -2.55857038e+00, -2.61219096e+00, ...,
          -2.46938586e+00, -2.52200556e+00, -2.52203703e+00]],

        [[ 1.16858854e+01,  1.16861229e+01,  1.20854702e+01, ...,
           1.08992252e+01,  1.12845449e+01,  1.12847786e+01],
         [ 1.19854193e+01,  1.22029753e+01,  1.24103947e+01, ...,
           1.14080744e+01,  1.16074972e+01,  1.17869263e+01],
         [ 1.77723351e+01,  1.77363186e+01,  1.77245960e+01, ...,
           1.75135403e+01,  1.76081600e+01,  1.77296162e+01],
         ...,
         [-4.45852137e+00, -4.45216084e+00, -4.46512365e+00, ...,
          -4.49451971e+00, -4.49552822e+00, -4.47297716e+00],
         [-2.47506595e+00, -2.51020455e+00, -2.55436301e+00, ...,
          -2.44886565e+00, -2.45971775e+00, -2.45823693e+00],
         [-2.37264442e+00, -2.37267828e+00, -2.41236496e+00, ...,
          -2.29071665e+00, -2.32201838e+00, -2.32203722e+00]]]],
      dtype=float32)
Coordinates:
  * time      (time) float64 149.5 161.5 173.5 185.5 ... 701.5 713.5 725.5 737.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    ensemble  (member) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
Dimensions without coordinates: member
Attributes:
    standard_name:  ahfs
    long_name:      var146 aka ahfs
    units:          W m-2
    code:           146
    table:          128

In [6]:
dat = np.mean(annual_prediction,axis=0)
dif = annual_prediction

In [21]:
dropped = dif.stack(feature=("member","lat","lon")).dropna(dim="feature")
detrend = signal.detrend(dropped ,axis=0)
feature = dropped .coords["feature"]
time = dropped .coords["time"]
detrend = xr.DataArray(detrend, dims = ["time","feature"], coords = {"time":time,"feature":feature}).unstack()

In [22]:
detrend.to_netcdf("/work/uo1075/u241321/data/ahfs_1970-2019_assi_dt.nc")

In [23]:
# full_value
dif.to_netcdf("/work/uo1075/u241321/data/ahfs_1970-2019_assi_full_value.nc")

In [7]:
aa = annual_prediction - dat
aa.to_netcdf("/work/uo1075/u241321/data/ahfs_1970-2019_assi.nc")


In [8]:
aa

<xarray.DataArray 'ahfs' (time: 50, member: 16, lat: 180, lon: 360)>
array([[[[-5.02461433e-01, -5.02483368e-01, -5.43006897e-01, ...,
          -4.12972450e-01, -4.56985474e-01, -4.57011223e-01],
         [-5.25407791e-01, -5.28234482e-01, -5.46660423e-01, ...,
          -4.58582878e-01, -4.94173050e-01, -5.14054298e-01],
         [-8.81364822e-01, -8.79899979e-01, -8.84275436e-01, ...,
          -9.25437927e-01, -9.85258102e-01, -9.16488647e-01],
         ...,
         [ 1.48299217e-01,  1.54776335e-01,  1.28763318e-01, ...,
           1.86846733e-01,  1.59664392e-01,  1.49929047e-01],
         [ 1.72952563e-01,  1.91502541e-01,  2.07650453e-01, ...,
           1.72305524e-01,  1.78902507e-01,  1.68752462e-01],
         [ 1.96919948e-01,  1.96931422e-01,  2.07413644e-01, ...,
           1.70388669e-01,  1.74024582e-01,  1.74033388e-01]],

        [[-4.12652016e-01, -4.12667274e-01, -4.47633743e-01, ...,
          -3.47128868e-01, -3.76793861e-01, -3.76812935e-01],
         [-4.39524651e-01, -4.49440002e-01, -4.63889122e-01, ...,
          -3.91557693e-01, -4.14916039e-01, -4.28017616e-01],
         [-1.00211334e+00, -9.48848724e-01, -8.82658005e-01, ...,
          -8.58963013e-01, -9.01988983e-01, -9.86484528e-01],
...
          -1.66394639e+00, -1.65244126e+00, -1.62546372e+00],
         [-1.64349592e+00, -1.62330830e+00, -1.59152544e+00, ...,
          -1.64615798e+00, -1.65329802e+00, -1.65349233e+00],
         [-1.61980534e+00, -1.61978507e+00, -1.61781931e+00, ...,
          -1.64672732e+00, -1.63975632e+00, -1.63975525e+00]],

        [[ 1.41277313e-01,  1.41284943e-01,  1.55047417e-01, ...,
           1.41473770e-01,  1.29715919e-01,  1.29715919e-01],
         [ 3.89375687e-02,  5.47437668e-02,  6.59093857e-02, ...,
           8.11262131e-02,  7.27281570e-02,  4.49333191e-02],
         [-9.19399261e-01, -1.07908630e+00, -1.19560242e+00, ...,
          -7.59870529e-01, -8.16703796e-01, -8.31935883e-01],
         ...,
         [-1.81618762e+00, -1.78772640e+00, -1.77788162e+00, ...,
          -1.92377591e+00, -1.90209651e+00, -1.85431099e+00],
         [-1.25289190e+00, -1.25773251e+00, -1.27445996e+00, ...,
          -1.32200134e+00, -1.30271912e+00, -1.26788723e+00],
         [-1.24323177e+00, -1.24323213e+00, -1.22972667e+00, ...,
          -1.27298033e+00, -1.24859536e+00, -1.24857950e+00]]]],
      dtype=float32)
Coordinates:
  * time      (time) float64 149.5 161.5 173.5 185.5 ... 701.5 713.5 725.5 737.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    ensemble  (member) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
Dimensions without coordinates: member